In [2]:
from ultralytics.data.annotator import auto_annotate

auto_annotate(data="Plant-Pathology-Anotation-7/train/images/", det_model="runs/detect/train3/weights/best.pt", sam_model="sam2_b.pt", output_dir="./SAM2/train")


image 1/1557 /home/abiyamf/Thesis/Plant-Pathology-Anotation-7/train/images/healthy_train_100_jpg.rf.70a71d45c044dfee48c83c1d0512ec62.jpg: 448x640 4 leaf_healthys, 38.0ms
image 2/1557 /home/abiyamf/Thesis/Plant-Pathology-Anotation-7/train/images/healthy_train_101_jpg.rf.dc3bab9ba484fb3950315355eab85338.jpg: 448x640 11 leaf_healthys, 29.2ms
image 3/1557 /home/abiyamf/Thesis/Plant-Pathology-Anotation-7/train/images/healthy_train_102_jpg.rf.88faddf5101d69a9f3d97c4e795a5c7d.jpg: 448x640 2 leaf_healthys, 30.0ms
image 4/1557 /home/abiyamf/Thesis/Plant-Pathology-Anotation-7/train/images/healthy_train_103_jpg.rf.e81cdde2e71064c27eda96d9d0eee3c8.jpg: 448x640 3 leaf_healthys, 30.5ms
image 5/1557 /home/abiyamf/Thesis/Plant-Pathology-Anotation-7/train/images/healthy_train_105_jpg.rf.b90a827a1f68a430f94d275d15f4248c.jpg: 448x640 4 leaf_healthys, 28.7ms
image 6/1557 /home/abiyamf/Thesis/Plant-Pathology-Anotation-7/train/images/healthy_train_108_jpg.rf.ab98bb8cca726540e99c84de793281a6.jpg: 448x640 3 

In [3]:
import cv2
import numpy as np
import os
from tqdm import tqdm

def visualize_segmentation(image_path, label_path, output_path):
    """
    Memvisualisasikan anotasi segmentasi YOLO pada sebuah gambar dan menyimpannya.
    (Fungsi inti dari jawaban sebelumnya, sedikit dimodifikasi untuk proses massal)
    """
    img = cv2.imread(image_path)
    if img is None:
        print(f"Warning: Gagal membaca file gambar, dilewati: {image_path}")
        return
        
    h, w, _ = img.shape
    overlay = img.copy()

    with open(label_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            class_id = int(parts[0])
            
            # Denormalisasi koordinat
            points = np.array([float(p) for p in parts[1:]]).reshape(-1, 2)
            points[:, 0] *= w
            points[:, 1] *= h
            points = points.astype(np.int32)

            # Gambar poligon (warna hijau terang)
            color = (36, 255, 12)
            cv2.fillPoly(overlay, [points], color)
            
    # Blend gambar asli dengan overlay anotasi
    alpha = 0.4  # Tingkat transparansi
    final_image = cv2.addWeighted(overlay, alpha, img, 1 - alpha, 0)
    
    # Gambar ulang garis tepi agar lebih jelas setelah di-blend
    with open(label_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            points = np.array([float(p) for p in parts[1:]]).reshape(-1, 2)
            points[:, 0] *= w
            points[:, 1] *= h
            points = points.astype(np.int32)
            color = (36, 255, 12)
            cv2.polylines(final_image, [points], isClosed=True, color=color, thickness=2)

    # Simpan hasil
    cv2.imwrite(output_path, final_image)

# --- FUNGSI UTAMA UNTUK MEMPROSES SEMUA FOLDER ---
def process_folders(image_dir, label_dir, output_dir):
    """
    Melakukan iterasi pada semua gambar di image_dir, mencari label yang sesuai
    di label_dir, dan menyimpan hasilnya ke output_dir.
    """
    # 1. Buat folder output jika belum ada
    os.makedirs(output_dir, exist_ok=True)
    print(f"Folder output '{output_dir}' telah siap.")

    # Ambil daftar semua file gambar
    image_files = [f for f in os.listdir(image_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    
    if not image_files:
        print(f"Tidak ada file gambar yang ditemukan di: {image_dir}")
        return

    print(f"Ditemukan {len(image_files)} gambar. Memulai proses...")
    
    # 2. Loop melalui setiap file gambar dengan progress bar
    for image_filename in tqdm(image_files, desc="Memproses Gambar"):
        # Buat path lengkap untuk file gambar
        image_path = os.path.join(image_dir, image_filename)
        
        # 3. Cari file label yang sesuai
        # Misal: gambar 'image1.jpg' akan mencari label 'image1.txt'
        base_filename = os.path.splitext(image_filename)[0]
        label_filename = base_filename + ".txt"
        label_path = os.path.join(label_dir, label_filename)
        
        # Path untuk menyimpan gambar hasil
        output_path = os.path.join(output_dir, image_filename)
        
        # Periksa apakah file label ada
        if os.path.exists(label_path):
            # Jika ada, panggil fungsi visualisasi
            visualize_segmentation(image_path, label_path, output_path)
        else:
            # Jika tidak ada, cetak peringatan (opsional, agar tidak terlalu ramai)
            # print(f"Warning: Tidak ditemukan file label untuk '{image_filename}' di '{label_dir}'. Dilewati.")
            pass # Lewati file ini jika tidak ada anotasi

    print("Selesai! Semua gambar telah diproses.")


if __name__ == '__main__':
    # Tentukan direktori Anda di sini
    IMAGE_DIRECTORY = "Plant-Pathology-Anotation-7/valid/images/"
    LABEL_DIRECTORY = "SAM2/"
    OUTPUT_DIRECTORY = "segmentasi/"
    
    process_folders(IMAGE_DIRECTORY, LABEL_DIRECTORY, OUTPUT_DIRECTORY)

Folder output 'segmentasi/' telah siap.
Ditemukan 173 gambar. Memulai proses...


Memproses Gambar: 100%|██████████| 173/173 [00:02<00:00, 59.35it/s]

Selesai! Semua gambar telah diproses.
